In [1]:
# !pip install openpyxl

In [2]:
import pandas as pd
import numpy as np
import os
import re
import requests
from urllib.parse import urlparse
import shutil

import utils as utl

initial_df = pd.read_csv('../../public/data/digimons.csv')
in_game_digimons_df = pd.read_excel('../data/Time Stranger Complete Digimon List.xlsx')

in_game_dig_names = in_game_digimons_df['Name'].tolist()

## 1. Tratamentos na tabela

In [3]:
# Remoção de coluna 'type' que só tem valor NaN:
df = utl.remove_type_column(initial_df)

# Normalizando a coluna 'name':
df = utl.normliza_column_name(df)

# def normalize_name(name):
df['name'] = df['name'].apply(utl.normalize_name)
df = df[df['name'] != 'Agumon Black']
df = df.drop_duplicates(subset='name', keep='first').reset_index(drop=True)

# # Normalizando os valores da coluna 'level':
df['level'] = df['level'].apply(utl.normalize_level)

# Normalizando a coluna 'attribute':
df['attribute'] = df['attribute'].apply(utl.normalize_attribute)

# Normalizando a coluna 'family':
df['family'] = df['family'].apply(utl.normalize_family)

# Normalizando 'prior_forms':
df["prior_forms"] = df["prior_forms"].apply(utl.normalize_prior_forms)

# Normalizando as colunas 'next_forms' e 'lateral_next_forms':
df["next_forms"] = df["next_forms"].apply(utl.format_list_column)
df["lateral_next_forms"] = df["lateral_next_forms"].apply(utl.format_list_column)

# Normalizando a coluna'digifuse_forms':
df['digifuse_forms'] = df['digifuse_forms'].apply(utl.clean_digifuse)

# Normalizando a coluna 'attacks':
df['attacks'] = df['attacks'].apply(utl.normalize_attacks)

# Separando palavras com letras maiúsculas:
df['name'] = df['name'].str.replace(r'(?<=[a-z])(?=[A-Z])', ' ', regex=True)

# Adicionando coluna 'source':
names = in_game_dig_names

# Normalizando a coluna de 'attacs':
attacks_list = []

for i in range(df.shape[0]):

    splitted_list = df['attacks'].iloc[i].split('|')
    for word in splitted_list:
        attacks_list.append(word)
        
while "Unknown" in attacks_list:
    attacks_list.remove("Unknown")


## 2.3 Criar dataframe separado, só com os digimons do jogo

In [4]:
# # Criar dataframe base com todos os nomes da lista
# base_df = pd.DataFrame({'name': names})

# # Fazer merge com o df original
# merged_df = base_df.merge(df, on='name', how='left')

# # Coluna included: True se achou (não nulo), False caso contrário
# merged_df['included'] = merged_df['name'].notna()

# # Colunas exceto 'name' e 'included'
# cols_check = merged_df.columns.difference(["name", "included"])

# # Se todos os valores (exceto name/included) forem NaN, marcar included = False
# mask_missing = merged_df[cols_check].isna().all(axis=1)
# merged_df.loc[mask_missing, "included"] = False

# # Substitui todos os NaN por "Unknown"
# merged_df = merged_df.fillna("Unknown")
# merged_df['image'] = merged_df['image'].replace("Unknown", "")

# # Substituindo o endereço das imagens pelas imagens locais:
# origin = './imagens_digimon/'
# target = './new_digimon_images'

# # Cria pasta destino se não existir
# os.makedirs(target, exist_ok=True)

# # Lista de nomes do merged_df
# nomes_desejados = merged_df["name"].tolist()

# # Normalizar nomes para comparação (ignorar maiúsculas/minúsculas, espaços etc.)
# nomes_desejados_normalizados = {nome.lower().replace(" ", "_") for nome in nomes_desejados}

# # Loop pelos arquivos na pasta original
# for arquivo in os.listdir(origin):
#     nome_base, ext = os.path.splitext(arquivo)
#     if nome_base.lower().replace(" ", "_") in nomes_desejados_normalizados:
#         origem_arquivo = os.path.join(origin, arquivo)
#         destino_arquivo = os.path.join(target, arquivo)
#         shutil.copy2(origem_arquivo, destino_arquivo)

# # Pasta onde estão as imagens filtradas
# pasta_imagens = target

# # Criar dicionário: {nome_base: caminho_completo}
# mapa_imagens = {
#     os.path.splitext(f)[0].lower().replace("_", " "): os.path.join(pasta_imagens, f)
#     for f in os.listdir(pasta_imagens)
#     if f.lower().endswith((".jpg", ".jpeg", ".png"))
# }

# # Substituir coluna 'image' pelos paths locais (ou "Unknown" se não encontrado)
# merged_df["image"] = merged_df["name"].str.lower().map(mapa_imagens).fillna("no image")

# if 'id' in merged_df.columns:
#     merged_df['id'] = range(1, len(merged_df) + 1)

# # Separa um df com apenas os digimons que ficarma faltando:
# merged_df[merged_df['included'] == False].to_excel('../data/missing_digimons.xlsx', index=False)

# merged_df = merged_df[['id','name',
#  'image',
#  'level',
#  'attribute',
#  'family',
#  'prior_forms',
#  'next_forms',
#  'lateral_next_forms',
#  'digifuse_forms',
#  'attacks',
#  'included']]

# # Reformat image paths:
# merged_df['image'] = merged_df['image'].astype(str).str.strip().str.strip('"').str.strip("'")
# merged_df['image'] = merged_df['image'].str.split(r'[\\/]').str[-1]
# merged_df['image'] = '/digimons_images/' + merged_df['image']

## 2. Executando a versão final do DF e salvando em .csv

In [5]:
# merged_df.to_csv('../../public/ingame_digimons.csv', index=False)

## 3. Substituindo digimons sem imagem por imagem genérica "Who is this digimon"?

In [6]:
new_df = pd.read_csv("../../public/ingame_digimons copy.csv")
new_df.head(3)

,id,name,image,level,attribute,family,prior_forms,next_forms,lateral_next_forms,digifuse_forms,attacks,included
0,1,Aegiochusmon,/digimons_images/Aegiochusmon.jpg,Ultimate,Vaccine,Unknown,Aegiomon,Unknown,Unknown,Unknown,Unknown,True
1,2,Aegiochusmon Blue,/digimons_images/Aegiochusmon_Blue.jpg,Ultimate,Vaccine,Unknown,Aegiomon,Unknown,Unknown,Unknown,Unknown,True
2,3,Aegiochusmon Dark,/digimons_images/Aegiochusmon_Dark.jpg,Ultimate,Vaccine,Unknown,Aegiomon,Unknown,Unknown,Unknown,Unknown,True


In [7]:
# Substituir todas as ocorrências do caminho errado
new_df["image"] = new_df["image"].replace("/digimons_images/no image", "/digimons_images/Placeholder.jpg")

In [8]:
# Salvar de volta no arquivo (se quiser)
new_df.to_csv("../../public/ingame_digimons.csv", index=False)